In [1]:
import gradio as gr
import base64
import os
from openai import OpenAI  # Assuming Gemini is using OpenAI-compatible SDK
import pytesseract


e:\llm\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("gemini_api_kay", "r") as file:
    api_key_gem = file.read().strip()

os.environ["API_KEY_GEM"] = api_key_gem

In [5]:
'''
# Convert image to base64 string
def image_to_base64(image):
    buffered = image.convert("RGB")
    with open("temp.png", "wb") as f:
        buffered.save(f, format="PNG")
    with open("temp.png", "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")
'''
#Scrapping image and getting OCR data
def image_to_ocr(image):
    try:
        if not isinstance(image, Image.Image):
            return "Invalid image format"
        return pytesseract.image_to_string(image)
    except Exception as e:
        return f"Error extracting text: {str(e)}"

In [6]:
'''
before_text = image_to_ocr(before_img)
after_text = image_to_ocr(after_img)
'''

'\nbefore_text = image_to_ocr(before_img)\nafter_text = image_to_ocr(after_img)\n'

In [7]:
client=OpenAI(
    api_key=os.environ["API_KEY_GEM"],
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = "gemini-2.5-flash-preview-05-20"

def compare_dashboards_ocr_only(before_img, after_img):
    if before_img is None or after_img is None:
        return "❗ Please upload both BEFORE and AFTER dashboard images."
    
    before_text = image_to_ocr(before_img)
    after_text = image_to_ocr(after_img)
    
    messeges = [
        {
            "role": "system",
            "content": """You are an expert data analyst assistant.
Analyze the two provided OCR data of images of a data analyst dashboard.
The first OCR image text is the 'BEFORE' version.
The second OCR image text is the 'AFTER' version.

Please carefully compare them and provide a detailed, itemized list of the changes. Focus on:
1. Changes in chart types, data displayed within charts (e.g., bar heights, line trends, pie slice proportions, new data series, changes in axis scales).
2. Modifications to Key Performance Indicator (KPI) values or any other numerical data presented. If a number changed, state the old and new value if discernible.
3. Added, removed, or repositioned visual elements (e.g., charts, tables, text boxes, filters, legends, buttons).
4. Alterations in titles, labels, or annotations.
5. Significant changes in color schemes or overall styling if they impact readability or meaning.
6. Changes in dashboard layout or structure.
7. Determine if there are any filters present like slicers, pull downs and drill throughs and provide valuable opinions whether the change may or may not be affected because of them.
8. Pay close attention to subtle changes in granularity (e.g., drill-down or drill-up actions in charts). This can appear as changes in axis labels (e.g., year vs. quarter), data grouping, or breadcrumb navigations at the top of charts.


Be as specific and factual as possible."""
        },
        {"role":"user",
         "content":[
             {"type":"text","text": f"Before Dashboard Text{before_text}"},
             {"type":"text","text": f"After Dashboard Text{after_text}"}
             ]
             }
         ]
    
    responces = client.chat.completions.create(
    model= model,
    reasoning_effort="high",
    messages= messeges
    )

    return responces.choices[0].message.content


# Gradio UI
gr.Interface(
    fn=compare_dashboards_ocr_only,
    inputs=[
        gr.Image(type="pil", label="BEFORE Dashboard"),
        gr.Image(type="pil", label="AFTER Dashboard")
    ],
    outputs=gr.Textbox(label="Gemini Analysis Output"),
    title="Power BI Dashboard Change Detector",
    description="Upload BEFORE and AFTER dashboard screenshots to get a detailed comparison using Gemini LLM."
).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [ ]:
'''
# Gemini Client Initialization
client = OpenAI(
    api_key=os.environ["API_KEY_GEM"],
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Compare images using Gemini
def compare_dashboards(before_img, after_img):
    if before_img is None or after_img is None:
        return "Please upload both BEFORE and AFTER images."

    before_b64 = image_to_base64(before_img)
    after_b64 = image_to_base64(after_img)

    messages = [
        {
            "role": "system",
            "content": """You are an expert data analyst assistant.
Analyze the two provided images of a data analyst dashboard.
The first image is the 'BEFORE' version.
The second image is the 'AFTER' version.

Please carefully compare them and provide a detailed, itemized list of the changes. Focus on:
1. Changes in chart types, data displayed within charts (e.g., bar heights, line trends, pie slice proportions, new data series, changes in axis scales).
2. Modifications to Key Performance Indicator (KPI) values or any other numerical data presented. If a number changed, state the old and new value if discernible.
3. Added, removed, or repositioned visual elements (e.g., charts, tables, text boxes, filters, legends, buttons).
4. Alterations in titles, labels, or annotations.
5. Significant changes in color schemes or overall styling if they impact readability or meaning.
6. Changes in dashboard layout or structure.
7. Determine if there are any filters present like slicers, pull downs and drill throughs and provide valuable opinions whether the change may or may not be affected because of them.
8. Pay close attention to subtle changes in granularity (e.g., drill-down or drill-up actions in charts). This can appear as changes in axis labels (e.g., year vs. quarter), data grouping, or breadcrumb navigations at the top of charts.


Be as specific and factual as possible."""
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "There are two images before and after of a power bi dashboard, help me identify the changes"},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{before_b64}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{after_b64}"}}
            ]
        }
    ]

    response = client.chat.completions.create(
        model="gemini-2.5-flash-preview-05-20",
        reasoning_effort="high",
        messages=messages
    )

    return response.choices[0].message.content

# Gradio UI
gr.Interface(
    fn=compare_dashboards,
    inputs=[
        gr.Image(type="pil", label="BEFORE Dashboard"),
        gr.Image(type="pil", label="AFTER Dashboard")
    ],
    outputs=gr.Textbox(label="Gemini Analysis Output"),
    title="Power BI Dashboard Change Detector",
    description="Upload BEFORE and AFTER dashboard screenshots to get a detailed comparison using Gemini LLM."
).launch()
'''